In [1]:
%config IPCompleter.use_jedi = False
%pdb off
%load_ext autoreload
%autoreload 3
# %matplotlib inline
%matplotlib qt5
import mne
mne.viz.set_browser_backend("qt")  # or "matplotlib"
mne.set_config("MNE_BROWSER_BACKEND", "qt")  # or "matplotlib"
%gui qt

import IPython

# Jupyter-lab enable printing for any line on its own (instead of just the last one in the cell)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


Automatic pdb calling has been turned OFF
Using qt as 2D backend.



# Use MNE to load and analyze saved EEG and Motion recordings


In [8]:
import time
import re
from datetime import datetime, timezone

import uuid
from copy import deepcopy
from typing import Dict, List, Tuple, Optional, Callable, Union, Any
from nptyping import NDArray
from matplotlib import pyplot as plt

from pathlib import Path
import numpy as np
import pandas as pd
from numpy.typing import NDArray

import mne
from mne import set_log_level
from copy import deepcopy
import mne

from mne.io import read_raw

datasets = []
# mne.viz.set_browser_backend("Matplotlib")
mne.viz.set_browser_backend("qt")

from mne_lsl.player import PlayerLSL as Player
from mne_lsl.stream import StreamLSL as Stream

from phoofflineeeganalysis.analysis.MNE_helpers import MNEHelpers
from phoofflineeeganalysis.analysis.historical_data import HistoricalData
from phoofflineeeganalysis.analysis.motion_data import MotionData
from phoofflineeeganalysis.analysis.EEG_data import EEGComputations, EEGData
from phoofflineeeganalysis.analysis.anatomy_and_electrodes import ElectrodeHelper
# from ..EegProcessing import bandpower
# from phoofflineeeganalysis.EegProcessing import analyze_eeg_trends
from phoofflineeeganalysis.EegVisualization import VisHelpers
from phoofflineeeganalysis.analysis.SavedSessionsProcessor import SavedSessionsProcessor, SessionModality, DataModalityType

set_log_level("WARNING")


# db_root_path = Path('/content/drive/MyDrive/Databases').resolve()
db_root_path = Path(r'E:/Dropbox (Personal)/Databases').resolve()
assert db_root_path.exists(), f"'{db_root_path.as_posix()}' does not exist!"

# eeg_recordings_file_path: Path = Path(r'E:/Dropbox (Personal)/Databases/UnparsedData/EmotivEpocX_EEGRecordings/fif').resolve()
# headset_motion_recordings_file_path: Path = Path(r'E:/Dropbox (Personal)/Databases/UnparsedData/EmotivEpocX_EEGRecordings/MOTION_RECORDINGS/fif').resolve()

# assert eeg_recordings_file_path.exists()
# assert headset_motion_recordings_file_path.exists()

eeg_recordings_file_path: Path = db_root_path.joinpath('UnparsedData/EmotivEpocX_EEGRecordings/fif').resolve()
flutter_eeg_recordings_file_path: Path = db_root_path.joinpath('UnparsedData/EmotivEEG_FlutterRecordings').resolve()
flutter_motion_recordings_file_path: Path = db_root_path.joinpath('UnparsedData/EmotivEEG_FlutterRecordings/MOTION_RECORDINGS').resolve()
flutter_GENERIC_recordings_file_path: Path = db_root_path.joinpath('UnparsedData/EmotivEEG_FlutterRecordings/GENERIC_RECORDINGS').resolve()

headset_motion_recordings_file_path: Path = db_root_path.joinpath('UnparsedData/EmotivEpocX_EEGRecordings/MOTION_RECORDINGS/fif').resolve()
WhisperVideoTranscripts_LSL_Converted = db_root_path.joinpath('UnparsedData/WhisperVideoTranscripts_LSL_Converted').resolve()
pho_log_to_LSL_recordings_path: Path = db_root_path.joinpath('UnparsedData/PhoLogToLabStreamingLayer_logs').resolve()
## These contain little LSL .fif files with names like: '20250808_062814_log.fif',

eeg_analyzed_parent_export_path = db_root_path.joinpath('AnalysisData/MNE_preprocessed').resolve()
pickled_data_path = db_root_path.joinpath('AnalysisData/MNE_preprocessed/PICKLED_COLLECTION').resolve()
assert pickled_data_path.exists()


# n_most_recent_sessions_to_preprocess: int = None # None means all sessions
# n_most_recent_sessions_to_preprocess: int = 35
# n_most_recent_sessions_to_preprocess: int = 5
n_most_recent_sessions_to_preprocess = None


'qt'

# SavedSessionProcessor

In [9]:

sso: SavedSessionsProcessor = SavedSessionsProcessor(eeg_recordings_file_path=eeg_recordings_file_path,
                                                     headset_motion_recordings_file_path=headset_motion_recordings_file_path, WhisperVideoTranscripts_LSL_Converted_file_path=WhisperVideoTranscripts_LSL_Converted, pho_log_to_LSL_recordings_path=pho_log_to_LSL_recordings_path,
                                                    eeg_analyzed_parent_export_path=eeg_analyzed_parent_export_path, 
                                                     n_most_recent_sessions_to_preprocess=n_most_recent_sessions_to_preprocess, 
                                                    should_load_data=True, should_load_preprocessed=False,
                                                    #  should_load_data=True, should_load_preprocessed=True,
													)

# 2025-09-18 - LabRecorder XDF Imports

In [10]:
from phoofflineeeganalysis.analysis.EEG_data import EEGData
from phoofflineeeganalysis.analysis.MNE_helpers import DatasetDatetimeBoundsRenderingMixin, RawArrayExtended, RawExtended, up_convert_raw_objects, up_convert_raw_obj
from phoofflineeeganalysis.analysis.SavedSessionsProcessor import LabRecorderXDF, unwrap_single_element_listlike_if_needed


lab_recorder_output_path = Path(r"E:\Dropbox (Personal)\Databases\UnparsedData\LabRecorderStudies\sub-P001").resolve()
assert lab_recorder_output_path.exists()

labRecorder_PostProcessed_path: Path = sso.eeg_analyzed_parent_export_path.joinpath(f'LabRecorder_PostProcessed')
labRecorder_PostProcessed_path.mkdir(exist_ok=True)

should_write_final_merged_eeg_fif: bool = True
# should_write_final_merged_eeg_fif: bool = False
_out_eeg_raw, _out_xdf_stream_infos_df, lab_recorder_xdf_files = LabRecorderXDF.load_and_process_all(lab_recorder_output_path=lab_recorder_output_path, 
                                                                                                     labRecorder_PostProcessed_path=labRecorder_PostProcessed_path, should_write_final_merged_eeg_fif=should_write_final_merged_eeg_fif)
xdf_dataset_indicies = np.unique(deepcopy(_out_xdf_stream_infos_df).reset_index(drop=False, inplace=False)['xdf_dataset_idx'].to_numpy())
n_unique_xdf_datasets: int = len(xdf_dataset_indicies)
print(f'n_unique_xdf_datasets: {n_unique_xdf_datasets}')


trying to process XDF file 0/39: "E:/Dropbox (Personal)/Databases/UnparsedData/LabRecorderStudies/sub-P001/LabRecorder_2025-09-10T140840.440Z_eeg.xdf"...
	 failed with error: 1
	skipping file.
trying to process XDF file 1/39: "E:/Dropbox (Personal)/Databases/UnparsedData/LabRecorderStudies/sub-P001/LabRecorder_2025-09-10T153731.079Z_eeg.xdf"...


Stream 3: Calculated effective sampling rate 32.0162 Hz is different from specified rate 16.0000 Hz.


saving finalized EEG data out to "E:/Dropbox (Personal)/Databases/AnalysisData/MNE_preprocessed/LabRecorder_PostProcessed/2025-09-10T15-37-31.fif"


C:\Users\pho\repos\EmotivEpoc\PhoOfflineEEGAnalysis\src\phoofflineeeganalysis\analysis\SavedSessionsProcessor.py:981: RuntimeWarning: This filename (E:\Dropbox (Personal)\Databases\AnalysisData\MNE_preprocessed\LabRecorder_PostProcessed\2025-09-10T15-37-31.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  eeg_raw.save(a_lab_recorder_filepath, overwrite=True)


trying to process XDF file 2/39: "E:/Dropbox (Personal)/Databases/UnparsedData/LabRecorderStudies/sub-P001/LabRecorder_2025-09-10T165236.402Z_eeg.xdf"...


...

trying to process XDF file 37/39: "E:/Dropbox (Personal)/Databases/UnparsedData/LabRecorderStudies/sub-P001/LabRecorder_rMBP_2025-09-11T203807.731Z.xdf"...


Stream 3: Calculated effective sampling rate 32.0160 Hz is different from specified rate 16.0000 Hz.


	 failed with error: not enough values to unpack (expected 2, got 1)
	skipping file.
trying to process XDF file 38/39: "E:/Dropbox (Personal)/Databases/UnparsedData/LabRecorderStudies/sub-P001/LabRecorder_rMBP_2025-09-11T210420.828Z.xdf"...


Stream 4: Calculated effective sampling rate 32.0167 Hz is different from specified rate 16.0000 Hz.
Stream 5: Calculated effective sampling rate 32.0166 Hz is different from specified rate 16.0000 Hz.


	 failed with error: orig_time should be the same to add/concatenate 2 annotations (got 2025-09-11 21:04:20+00:00 != None)
	skipping file.
n_unique_xdf_datasets: 25


In [11]:
from phoofflineeeganalysis.analysis.SavedSessionsProcessor import XDFDataStreamAccessor


_out_xdf_stream_infos_df: pd.DataFrame = XDFDataStreamAccessor.init_from_results(_out_xdf_stream_infos_df=_out_xdf_stream_infos_df, active_only_out_eeg_raws=_out_eeg_raw) # [_out_xdf_stream_infos_df['name'] == 'Epoc X']
_out_xdf_stream_infos_df

,name,fs,n_samples,n_channels,stream_id,effective_srate,hostname,source_id,channel_format,type,...,recording_day_date,duration_sec,created_at_dt,created_at_rel,first_timestamp_dt,first_timestamp_rel,last_timestamp_dt,last_timestamp_rel,duration_sec_check,duration_rel
xdf_dataset_idx,,,,,,,,,,,,,,,,,,,,,
0,Epoc X eQuality,128.0,333030,14,1,128.373095,EmotivVMApogee,Emotiv Epoc X_8_36353636353635363636373536353537,float32,Raw,...,2025-09-10 00:00:00+00:00,0 days 00:43:21.796875,2025-09-10 17:15:45.777074+00:00,0.068227,2025-09-10 17:19:57.910380800+00:00,0.071145,2025-09-10 18:03:12.432241900+00:00,0.101174,0 days 00:43:14.521861100,0.030029
0,Epoc X,128.0,333028,14,2,128.373095,EmotivVMApogee,Emotiv Epoc X_8_36353636353635363636373536353537,float32,EEG,...,2025-09-10 00:00:00+00:00,0 days 00:43:21.781250,2025-09-10 17:15:45.788452800+00:00,0.068227,2025-09-10 17:19:57.927472+00:00,0.071145,2025-09-10 18:03:12.432253800+00:00,0.101174,0 days 00:43:14.504781800,0.030029
0,Epoc X Motion,16.0,83063,6,3,32.016169,EmotivVMApogee,Emotiv Epoc X_8_36353636353635363636373536353537,float32,SIGNAL,...,2025-09-10 00:00:00+00:00,0 days 01:26:31.437500,2025-09-10 17:15:45.798384700+00:00,0.068227,2025-09-10 17:19:57.951104100+00:00,0.071145,2025-09-10 18:03:12.449342+00:00,0.101174,0 days 00:43:14.498237900,0.030029
1,Epoc X Motion,16.0,5440,6,2,19.594720,rMBP-16-2023.local,Emotiv Epoc X_8_36353636353635363636373536353537,float32,SIGNAL,...,2025-09-11 00:00:00+00:00,0 days 00:05:40,2025-09-16 13:15:16.039368664+00:00,5.481505,NaT,NaN,NaT,NaN,NaT,NaN
1,Epoc X eQuality,128.0,21800,14,3,126.501594,rMBP-16-2023.local,Emotiv Epoc X_8_36353636353635363636373536353537,float32,Raw,...,2025-09-11 00:00:00+00:00,0 days 00:02:50.312500,2025-09-16 13:15:16.016466427+00:00,5.481505,NaT,NaN,NaT,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23,Epoc X,128.0,236038,14,3,128.400104,rMBP-PinkDot.local,Emotiv Epoc X_8_36353636353635363636373536353537,float32,EEG,...,2025-09-21 00:00:00+00:00,0 days 00:30:44.046875,2025-09-21 10:13:58.309235326+00:00,0.054367,2025-09-21 11:08:05.824650232+00:00,0.091954,2025-09-21 11:38:44.332823942+00:00,0.113233,0 days 00:30:38.508173710,0.021279
23,TextLogger,0.0,5,1,2,0.000000,Apogee,textlogger_001,string,Markers,...,2025-09-21 00:00:00+00:00,NaT,2025-09-21 10:43:17.263254200+00:00,0.074725,2025-09-21 11:14:23.115459500+00:00,0.096321,2025-09-21 11:41:36.181905700+00:00,0.115222,0 days 00:27:13.066446200,0.018901
24,Epoc X Motion,16.0,38219,6,2,27.559387,rMBP-PinkDot.local,Emotiv Epoc X_8_36353636353635363636373536353537,float32,SIGNAL,...,2025-09-22 00:00:00+00:00,0 days 00:39:48.687500,2025-09-24 00:02:47.827532591+00:00,1.102093,2025-09-24 00:04:04.804846451+00:00,1.102984,2025-09-24 00:26:40.282300291+00:00,1.118672,0 days 00:22:35.477453840,0.015688


In [ ]:
# from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.SpikeRasterWidgets.Spike2DRaster import Spike2DRaster, SynchronizedPlotMode

In [ ]:
_out_xdf_stream_infos_df

In [ ]:
len(_out_xdf_stream_infos_df)
_out_xdf_stream_infos_df.loc[0]

In [ ]:
_out_eeg_raw

# Run Batch Computations on `_out_eeg_raw`

In [ ]:
num_sessions: int = len(_out_eeg_raw)
num_sessions


results_dict = {}

for an_xdf_dataset_idx in np.arange(num_sessions):
    a_raw = deepcopy(_out_eeg_raw[an_xdf_dataset_idx])
    a_raw.down_convert_to_base_type()
    results_dict[an_xdf_dataset_idx] 

## Common Setup

In [ ]:
a_raw = deepcopy(_out_eeg_raw[-3])
a_raw.down_convert_to_base_type()

In [ ]:
## INPUT: fixed_len_epochs
freqs = np.arange(5., 40., 1.0)
# Define frequencies and number of cycles
# freqs = np.logspace(*np.log10([2, 40]), num=20)
n_cycles = freqs / 2.0 # A common approach is to use a fixed number of cycles or a value that increases with frequency.

freqs
n_cycles

In [ ]:
from pyphoplacecellanalysis.GUI.PyQtPlot.BinnedImageRenderingWindow import BasicBinnedImageRenderingWindow, LayoutScrollability


### Fixed Length Epoch-based:

In [ ]:
from phoofflineeeganalysis.analysis.MNE_helpers import RawExtended, RawArrayExtended

## INPUT: a_raw
fixed_len_epochs = mne.make_fixed_length_epochs(a_raw, duration=8, preload=True, reject_by_annotation=False)
fixed_len_epochs
	

In [ ]:
## INPUT: fixed_len_epochs
freqs = np.arange(5., 40., 1.0)

# Define frequencies and number of cycles
# freqs = np.logspace(*np.log10([2, 40]), num=20)
n_cycles = freqs / 2.0 # A common approach is to use a fixed number of cycles or a value that increases with frequency.

# n_cycles = np.linspace(3,10,len(freqs))


In [ ]:


# Compute time-frequency power using Morlet wavelets -- NO EPOCHS, RAW DATA
# power = a_raw.compute_tfr(method="morlet",
power = mne.time_frequency.tfr_morlet(
    fixed_len_epochs,
    freqs=freqs,
    n_cycles=n_cycles,
    return_itc=False,
    average=False, # Compute TFR on continuous data without averaging
    # average=True, # Compute TFR on continuous data without averaging
    decim=3, # Decimate the output for faster computation and smaller file size
    n_jobs=-1 # Use 1 core for this example
)
power

In [ ]:
## Array version:
from mne.time_frequency import tfr_array_morlet

# get raw data (channels × time)
data = fixed_len_epochs.get_data() # shape: (n_epochs, n_channels, n_times)
print(f'np.shape(data): {np.shape(data)}')
# run morlet on continuous data
power: NDArray = tfr_array_morlet(
    data=data,
    sfreq=fixed_len_epochs.info['sfreq'],
    freqs=freqs,
    n_cycles=n_cycles,
    output='power',
    decim=3,
    n_jobs=-1
)

# power shape: (n_epochs=1, n_channels, n_freqs, n_times)
power = np.squeeze(power) ## remove epoch
print(f'np.shape(power): {np.shape(power)}')

# np.shape(data): (593, 14, 1024)
# np.shape(power): (593, 14, 20, 342)

In [ ]:
import h5py

repo_data_folder_path = Path(r"C:\Users\pho\repos\EmotivEpoc\PhoLabStreamingReceiver\data").resolve()

h5_file_path = repo_data_folder_path.joinpath(f'2025-09-21_exported_sess.h5').resolve()

print(f'writing to "{h5_file_path.as_posix()}"')

# raw_data = a_raw.get_data() # (14, 607857)
# np.shape(raw_data)


In [ ]:
with h5py.File(h5_file_path, "w") as f:
    
    d0 = f.create_dataset("raw", data=raw_data)
    d0.attrs["dim_labels"] = ["channels", "time"]

    d1 = f.create_dataset("epoched", data=data)
    d1.attrs["dim_labels"] = ["epochs", "channels", "time"]

    d2 = f.create_dataset("power", data=power)
    d2.attrs["dim_labels"] = ["epochs", "channels", "time", "frequency"]


print(f'done! wrote to "{h5_file_path.as_uri()}"')

# ACTIVE: refine the active raws and compute them

In [12]:
from phoofflineeeganalysis.analysis.EEG_data import EEGComputations, EEGData
from phoofflineeeganalysis.PendingNotebookCode import batch_compute_all_eeg_datasets, render_all_spectograms_to_high_quality_pdfs, plot_all_spectograms
from phoofflineeeganalysis.PendingNotebookCode import plot_session_spectogram
## INPUTS: _out_eeg_raw
# Process only the last 5 datasets using 4 workers:
active_only_out_eeg_raws, results = batch_compute_all_eeg_datasets(eeg_raws=_out_eeg_raw, limit_num_items=150, max_workers = 4)

## OUTPUTS: active_only_out_eeg_raws, results
# 1m 19.8s for 25 sessions

In [ ]:
from phoofflineeeganalysis.analysis.SavedSessionsProcessor import XDFDataStreamAccessor

num_sessions: int = len(results)
num_sessions

# xdf_stream_infos_df: pd.DataFrame = XDFDataStreamAccessor.init_from_results(_out_xdf_stream_infos_df=_out_xdf_stream_infos_df, active_only_out_eeg_raws=active_only_out_eeg_raws)
xdf_stream_infos_df: pd.DataFrame = XDFDataStreamAccessor.init_from_results(_out_xdf_stream_infos_df=_out_xdf_stream_infos_df, active_only_out_eeg_raws=active_only_out_eeg_raws)
# xdf_stream_infos_df: pd.DataFrame = XDFDataStreamAccessor.init_from_results(_out_xdf_stream_infos_df=_out_xdf_stream_infos_df[_out_xdf_stream_infos_df['name'] == 'Epoc X'], active_only_out_eeg_raws=active_only_out_eeg_raws)
xdf_stream_infos_df

In [22]:
import hvplot.xarray

for a_result in results:
    spec_da = a_result['spectogram']['Sxx']


spec_da

<xarray.DataArray (channels: 14, freqs: 513, times: 1116)> Size: 64MB
array([[[2.00315679e+02, 7.27589085e+02, 1.60173939e+02, ...,
         1.76903195e+01, 1.99480317e+01, 1.32972980e+01],
        [1.00747044e+04, 8.39587570e+03, 5.80526569e+03, ...,
         1.52358788e+04, 1.42908480e+03, 2.42481162e+04],
        [4.12927376e+03, 2.05819956e+02, 3.53096825e+02, ...,
         1.35474018e+04, 1.83079975e+03, 2.26230589e+04],
        ...,
        [1.78493648e-03, 1.60401703e-03, 2.27386567e-03, ...,
         1.75445630e-03, 3.86508685e-04, 1.81143215e-03],
        [2.26209829e-03, 3.19858964e-03, 4.14797607e-03, ...,
         5.84998232e-03, 7.51164446e-04, 7.88447143e-04],
        [2.59222657e-03, 1.84897888e-04, 8.12754322e-04, ...,
         1.44774193e-04, 1.11505082e-05, 1.12541780e-03]],

       [[1.59517462e+02, 6.21686703e+02, 7.16576119e+02, ...,
         1.63040031e+01, 1.02624013e+00, 1.03092545e+01],
        [8.60310716e+03, 7.83767396e+03, 4.62566000e+03, ...,
         2.06641213e+04, 6.27989849e+03, 2.60237705e+04],
        [2.89950344e+03, 1.28615777e+02, 9.01935776e+02, ...,
         1.51406909e+04, 8.18281068e+03, 1.14144340e+04],
...
        [2.45279058e-03, 3.04500378e-03, 3.05249594e-03, ...,
         1.29152187e-03, 9.02103396e-04, 2.13528528e-04],
        [1.60750835e-03, 1.16413438e-03, 4.19578127e-03, ...,
         5.40649207e-03, 2.04946566e-04, 4.53623331e-04],
        [6.02676063e-04, 5.18761996e-04, 3.05152875e-03, ...,
         1.85759957e-05, 3.89687632e-04, 4.80253429e-04]],

       [[1.96922733e+02, 4.89241262e+02, 2.29559962e+02, ...,
         1.53709998e+01, 1.21780046e+01, 5.91726184e+00],
        [1.35742024e+04, 1.08012260e+04, 2.97176700e+03, ...,
         3.45057423e+04, 1.46148216e+04, 1.79761007e+04],
        [4.77560229e+03, 1.06005807e+03, 5.17943763e+02, ...,
         1.43409101e+04, 4.41178627e+03, 3.67096971e+03],
        ...,
        [9.59067249e-04, 6.33212512e-04, 2.89053804e-03, ...,
         1.12905709e-02, 5.03695778e-04, 1.29186057e-03],
        [1.22933279e-03, 1.84401246e-03, 3.97074639e-03, ...,
         2.12869169e-02, 2.32982305e-03, 6.27452924e-04],
        [1.26453163e-03, 2.12337802e-04, 7.65206596e-04, ...,
         2.09802004e-04, 7.99921558e-04, 7.82462444e-04]]])
Coordinates:
  * channels  (channels) <U3 168B 'AF3' 'F7' 'F3' 'FC5' ... 'F4' 'F8' 'AF4'
  * freqs     (freqs) float64 4kB 0.0 0.125 0.25 0.375 ... 63.75 63.88 64.0
  * times     (times) float64 9kB 4.0 8.0 12.0 ... 4.456e+03 4.46e+03 4.464e+03

In [ ]:
spec_da.mean(dim=(''

In [ ]:

# Generate an interactive spectrogram with a slider for the time dimension
hv_plot_slider = spec_da.hvplot(
    x='times',
    y='freqs',
    cmap='viridis',
    clim=(spec_da.min(), spec_da.max()), # Set color limits
    width=800,
    height=400,
    groupby='times', # This is the key argument
    title='Interactive Spectrogram with hvPlot Slider'
)

hv_plot_slider

# Save the linked layout to a self-contained HTML file
hv.save(
    linked_layout,
    'holoviz_spectrogram.html',
    backend='bokeh',
    # The resources parameter can be used to control how JS/CSS are included,
    # with 'INLINE' being the most portable option.
    resources='INLINE'
)



In [ ]:
from phoofflineeeganalysis.analysis.EEG_data import EEGComputations
from phoofflineeeganalysis.analysis.SavedSessionsProcessor import LabRecorderXDF

hdf5_out_path: Path = Path('E:/Dropbox (Personal)/Databases/AnalysisData/MNE_preprocessed').joinpath('2025-09-29_all_HDF.h5').resolve()

hdf5_out_path.unlink(missing_ok=True)

LabRecorderXDF.to_hdf(active_only_out_eeg_raws=active_only_out_eeg_raws, results=results, xdf_stream_infos_df=xdf_stream_infos_df, file_path=hdf5_out_path, root_key='/')


In [ ]:
flat_annotations = []
for an_xdf_dataset_idx in np.arange(num_sessions):
    a_raw = active_only_out_eeg_raws[an_xdf_dataset_idx]
    a_meas_date = a_raw.info.get('meas_date')
    a_raw_key: str = a_meas_date.strftime("%Y-%m-%d/%H-%M-%S") # '2025-09-22/21-35-47'
    a_result = results[an_xdf_dataset_idx]
    # EEGComputations.to_hdf(a_result=a_result, file_path=hdf5_out_path, root_key=f'/result/{a_raw_key}')
    # EEGComputations.to_hdf(a_result=a_result, file_path=f, root_key=f'/result/{a_raw_key}')
    # EEGComputations.perform_write_to_hdf(a_result=a_result, f=f, root_key=f'/result/{a_raw_key}')
    # a_stream_info = deepcopy(_out_xdf_stream_infos_df).loc[an_xdf_dataset_idx]    
    # print(f'i: {i}, a_meas_date: {a_meas_date}, a_stream_info: {a_stream_info}\n\n')
    # print(f'i: {an_xdf_dataset_idx}, a_meas_date: {a_meas_date}')
    an_annotations_df = a_raw.annotations.to_data_frame(time_format='datetime')
    an_annotations_df = an_annotations_df[an_annotations_df['description'] != 'BAD_motion']
    an_annotations_df['xdf_dataset_idx'] = an_xdf_dataset_idx
    an_annotations_df['xdf_raw_key'] = a_raw_key
    flat_annotations.append(an_annotations_df)

    ## Add metadata to a_raw and a_result
    # 2025-09-18/03-18-42

flat_annotations = pd.concat(flat_annotations, ignore_index=True)
flat_annotations

,onset,duration,description,xdf_dataset_idx,xdf_raw_key
0,2025-09-11 10:13:28.000027,0.0,sync,2,2025-09-11/10-13-28
1,2025-09-11 10:13:28.000030,0.0,1.0+,2,2025-09-11/10-13-28
2,2025-09-11 10:13:28.000030,0.0,pushes me up to 2.5+ for the day,2,2025-09-11/10-13-28
3,2025-09-11 10:13:28.000030,0.0,forgot I was recording EEG so I didn't record ...,2,2025-09-11/10-13-28
4,2025-09-11 15:45:49.000047,0.0,started lab recorder,3,2025-09-11/15-45-49
...,...,...,...,...,...
75,2025-09-22 21:35:47.000042,0.0,like I'm no longer up to the challenge now tha...,24,2025-09-22/21-35-47
76,2025-09-22 21:35:47.000042,0.0,I was so excited to do this before,24,2025-09-22/21-35-47
77,2025-09-22 21:35:47.000042,0.0,it seemed so promising,24,2025-09-22/21-35-47
78,2025-09-22 21:35:47.000044,0.0,RECORDING_STOPPED: 20250922_172926_log.xdf,24,2025-09-22/21-35-47


In [ ]:
import xarray as xr

cog_categorical_cats = ['cog_bad', 'cog_poor', 'cog_okay', 'cog_good', 'cog_great']

day_status_dict = {'2025-09-19/01-54-39': 'cog_poor',
'2025-09-18/15-23-08': 'cog_poor',
'2025-09-22/21-35-47': 'cog_okay',
'2025-09-19/20-51-18': 'cog_good',
'2025-09-18/03-18-42': 'cog_great',
'2025-09-22/21-35-47': 'cog_bad',
'2025-09-21/08-55-41': 'cog_bad',
}



In [ ]:

all_Sxx = []
for an_xdf_dataset_idx in np.arange(num_sessions):
    a_raw = active_only_out_eeg_raws[an_xdf_dataset_idx]
    a_meas_date = a_raw.info.get('meas_date')
    a_raw_key: str = a_meas_date.strftime("%Y-%m-%d/%H-%M-%S") # '2025-09-22/21-35-47'
    a_status = day_status_dict.get(a_raw_key, None)
    if a_status:
        ## a status
        a_result = results[an_xdf_dataset_idx]
        Sxx = a_result['spectogram']['Sxx']
        Sxx = Sxx.assign_attrs(cognitive_status=a_status) # xarray.DataArray - channels: 14, freqs: 513, times: 1116
        # Sxx ## TODO: save the xarray to a single file (as a list of xarrays for each `an_xdf_dataset_idx`
        Sxx = Sxx.expand_dims(session=[a_raw_key])
        all_Sxx.append(Sxx)

if all_Sxx:
    combined = xr.concat(all_Sxx, dim="session")
    # combined.to_netcdf("all_sessions_spectrograms.nc", engine="h5netcdf", encoding={"Sxx": {"zlib": True, "complevel": 5}})
    combined

In [63]:
import xarray as xr
import zarr
import numcodecs
import numpy as np
from pathlib import Path
from typing import Union

def load_sessions_from_zarr(zarr_path: Union[str, Path], verbose: bool = False) -> xr.Dataset:
    zarr_path = Path(zarr_path)
    if not zarr_path.exists():
        raise FileNotFoundError(f"Zarr store not found at {zarr_path}")

    store = zarr.DirectoryStore(zarr_path)
    root = zarr.open(store, mode="r")
    session_keys = list(root.group_keys())
    if verbose:
        print("zarr groups:", session_keys)
    if not session_keys:
        raise ValueError(f"No session groups found in the Zarr store at {zarr_path}")

    da_list: List[xr.DataArray] = []
    for key in session_keys:
        try:
            ds = xr.open_zarr(store, group=key, consolidated=True)
        except Exception as e:
            if verbose: print(f"skipping group {key}: open_zarr error: {e}")
            continue

        # choose the DataArray: prefer 'Sxx' but fall back to the first data_var
        if "Sxx" in ds.data_vars:
            da = ds["Sxx"]
            picked_name = "Sxx"
        elif len(ds.data_vars) == 1:
            picked_name = list(ds.data_vars)[0]
            da = ds[picked_name]
            if verbose: print(f"group {key}: no 'Sxx' var, picked '{picked_name}'")
        else:
            if verbose: print(f"skipping group {key}: no Sxx and multiple data_vars: {list(ds.data_vars)}")
            continue

        # normalize name to 'Sxx' for consistency
        if da.name != "Sxx":
            da = da.rename("Sxx")

        session_key = ds.attrs.get("session_key", key)
        cognitive_status = da.attrs.get("cognitive_status", "unknown")

        # Expand with session dim labeled by session_key
        da = da.expand_dims({"session": [session_key]})

        # Assign cognitive_status as a coordinate on session dim
        da = da.assign_coords(cognitive_status=("session", [cognitive_status]))

        da_list.append(da)

    if not da_list:
        raise ValueError("No sessions with Sxx (or fallback data-var) found in Zarr store")

    combined = xr.concat(da_list, dim="session", join="outer")

    ds_combined = combined.to_dataset(name="Sxx")

    # session_key coord (same labels as session dim)
    sess_labels = [str(x) for x in ds_combined["session"].values.tolist()]
    ds_combined = ds_combined.assign_coords(session_key=("session", sess_labels))

    # ensure cognitive_status exists as session coord
    if "cognitive_status" not in ds_combined.coords:
        cs_vals = [da.coords.get("cognitive_status").values[0] if "cognitive_status" in da.coords else "unknown" for da in da_list]
        ds_combined = ds_combined.assign_coords(cognitive_status=("session", cs_vals))

    return ds_combined



def save_sessions_as_zarr(active_only_out_eeg_raws, results, day_status_dict, out_path="all_sessions.zarr"):
    """ write out the result (spectogram) from each session in the `day_status_dict` to the out_path file

    Usage:

        cog_categorical_cats = ['cog_bad', 'cog_poor', 'cog_okay', 'cog_good', 'cog_great']

        day_status_dict = {
            '2025-09-19/01-54-39': 'cog_poor',
            '2025-09-18/15-23-08': 'cog_poor',
            '2025-09-22/21-35-47': 'cog_okay',
            '2025-09-19/20-51-18': 'cog_good',
            '2025-09-18/03-18-42': 'cog_great',
            '2025-09-22/21-35-47': 'cog_bad',
            '2025-09-21/08-55-41': 'cog_bad',
        }


        out_path = Path("2025-09-29_all_sessions.zarr")
        out_path = save_sessions_as_zarr(active_only_out_eeg_raws=active_only_out_eeg_raws, results=results, day_status_dict=day_status_dict, out_path=out_path)

    """
    compressor = numcodecs.Blosc()
    store = zarr.DirectoryStore(out_path)
    num_sessions: int = len(active_only_out_eeg_raws)

    for idx in np.arange(num_sessions):
        a_raw = active_only_out_eeg_raws[idx]
        a_meas_date = a_raw.info.get('meas_date')
        a_raw_key = a_meas_date.strftime("%Y-%m-%d/%H-%M-%S")
        a_status = day_status_dict.get(a_raw_key, None)
        if not a_status:
            continue

        a_result = results[idx]
        Sxx = a_result['spectogram']['Sxx']
        Sxx = Sxx.assign_attrs(cognitive_status=a_status) # xarray.DataArray - channels: 14, freqs: 513, times: 1116

        ds = Sxx.to_dataset(name='Sxx')
        ds.attrs['session_key'] = a_raw_key

        encoding = {v: {'compressor': compressor} for v in ds.data_vars}
        ds.to_zarr(store, group=a_raw_key, mode='a', encoding=encoding)

    zarr.consolidate_metadata(store)
    return out_path


cog_categorical_cats = ['cog_bad', 'cog_poor', 'cog_okay', 'cog_good', 'cog_great']

day_status_dict = {
    '2025-09-19/01-54-39': 'cog_poor',
    '2025-09-18/15-23-08': 'cog_poor',
    '2025-09-22/21-35-47': 'cog_okay',
    '2025-09-19/20-51-18': 'cog_good',
    '2025-09-18/03-18-42': 'cog_great',
    '2025-09-22/21-35-47': 'cog_bad',
    '2025-09-21/08-55-41': 'cog_bad',
}


out_path = Path("2025-09-30_all_sessions.zarr")

# out_path = save_sessions_as_zarr(active_only_out_eeg_raws=active_only_out_eeg_raws, results=results, day_status_dict=day_status_dict, out_path=out_path)

## Immediately re-load from file
all_data = load_sessions_from_zarr(out_path, verbose=True)


zarr groups: ['2025-09-18', '2025-09-19', '2025-09-21', '2025-09-22']
skipping group 2025-09-18: no Sxx and multiple data_vars: []
skipping group 2025-09-19: no Sxx and multiple data_vars: []
skipping group 2025-09-21: no Sxx and multiple data_vars: []
skipping group 2025-09-22: no Sxx and multiple data_vars: []


ValueError: No sessions with Sxx (or fallback data-var) found in Zarr store

# Export Spectogram XArray Spectograms to Interactive Plots

In [54]:
import xarray as xr
import zarr
import panel as pn
import hvplot.xarray
import numpy as np
from pathlib import Path

# # --- 1. Data Loading Function ---

# --- 2. Setup the Interactive Dashboard ---
ZARR_FILE_PATH = Path("2025-09-29_all_sessions.zarr")

try:
    all_data = load_sessions_from_zarr(ZARR_FILE_PATH)
except (FileNotFoundError, ValueError) as e:
    pn.pane.Alert(f"Error loading data: {e}", alert_type='danger').servable()
else:
    pn.extension(sizing_mode="stretch_width")

    # --- 3. Create Interactive Widgets ---
    cog_statuses = sorted(np.unique(all_data.cognitive_status.values).tolist())
    channels = all_data.channels.values.tolist()

    status_select = pn.widgets.Select(name='Cognitive Status', options=['All'] + cog_statuses, value='All')
    session_select = pn.widgets.Select(name='Session')
    channel_select = pn.widgets.Select(name='Channel', options=channels)

    # --- 4. Dependent Widgets ---
    @pn.depends(status_select.param.value, watch=True)
    def _update_session_options(status):
        if status == 'All':
            sessions = sorted(all_data.session.values.tolist())
        else:
            filtered_data = all_data.where(all_data.cognitive_status == status, drop=True)
            sessions = sorted(filtered_data.session.values.tolist())
        session_select.options = sessions
        if sessions:
            session_select.value = sessions[0]

    # --- 5. Plot Function ---
    @pn.depends(session_select.param.value, channel_select.param.value)
    def plot_spectrogram(session_key, channel):
        if not session_key or channel is None:
            return pn.pane.Markdown("### Please select a session and channel to view the spectrogram.")
        spec_da = all_data['Sxx'].sel(session=session_key, channels=channel).squeeze()
        spec_da_db = 10 * np.log10(spec_da + 1e-10)
        spec_da_db.attrs['long_name'] = 'Power (dB)'
        plot = spec_da_db.hvplot.quadmesh(
            x='times', y='freqs', cmap='viridis', rasterize=True, height=500,
            title=f"Session: {session_key} | Channel: {channel}", xlabel="Time (s)", ylabel="Frequency (Hz)",
            clabel="Power Spectral Density (dB)"
        ).opts(responsive=True)
        return plot

    # --- 6. Dashboard Layout ---
    dashboard = pn.template.FastListTemplate(
        site="EEG Spectrogram Viewer",
        title="Interactive Session Analysis",
        sidebar=[status_select, session_select, channel_select],
        main=[plot_spectrogram],
        accent_base_color="#4A90E2",
        header_background="#4A90E2",
    )

    dashboard.servable()


BokehModel(combine_events=True, render_bundle={'docs_json': {'981b914a-07d5-4174-9167-e442d7dd50e8': {'version…

In [ ]:
import zarr
from zarr.storage import MemoryStore


In [ ]:
# Create nested groups and add arrays
root = zarr.group("data/example-3.zarr", attributes={'name': 'root'})
foo = root.create_group(name="foo")
bar = root.create_array(
    name="bar", shape=(100, 10), chunks=(10, 10), dtype="f4"
)
nodes = {'': root.metadata} | {k: v.metadata for k,v in root.members()}
print(nodes)

new_nodes = dict(zarr.create_hierarchy(store=MemoryStore(), nodes=nodes))
new_root = new_nodes['']
assert new_root.attrs == root.attrs

In [28]:

named_sessions_dict = {'dumb': -2,
					   'good': -6,
}

# dumb_session_idx: int = -2

for a_name, an_xdf_dataset_idx in named_sessions_dict.items():
    a_raw = active_only_out_eeg_raws[an_xdf_dataset_idx]
    a_meas_date: datetime = a_raw.info.get('meas_date')
    a_raw_key: str = a_meas_date.strftime("%Y-%m-%d/%H-%M-%S") # '2025-09-22/21-35-47'
    day_status_dict[a_raw_key] = a_name

day_status_dict

{'2025-09-19/01-54-39': 'cog_poor',
 '2025-09-18/15-23-08': 'cog_poor',
 '2025-09-22/21-35-47': 'cog_bad',
 '2025-09-19/20-51-18': 'good',
 '2025-09-18/03-18-42': 'cog_great',
 '2025-09-21/08-55-41': 'dumb'}

In [ ]:
active_only_out_eeg_raws[an_xdf_dataset_idx]

In [ ]:
from phoofflineeeganalysis.analysis.EEG_data import EEGComputations, EEGData

write_mode = 'r+'
if (not hdf5_out_path.exists()):
    write_mode = 'w'
    
write_mode = 'w'

with h5py.File(hdf5_out_path, write_mode) as f:
    for an_xdf_dataset_idx in np.arange(num_sessions):
        if (an_xdf_dataset_idx < 5):
            a_raw = active_only_out_eeg_raws[an_xdf_dataset_idx]
            a_meas_date = a_raw.info.get('meas_date')
            a_raw_key: str = a_meas_date.strftime("%Y-%m-%d/%H-%M-%S") # '2025-09-22/21-35-47'
            a_result = results[an_xdf_dataset_idx]
            # EEGComputations.to_hdf(a_result=a_result, file_path=hdf5_out_path, root_key=f'/result/{a_raw_key}')
            # EEGComputations.to_hdf(a_result=a_result, file_path=f, root_key=f'/result/{a_raw_key}')
            EEGComputations.perform_write_to_hdf(a_result=a_result, f=f, root_key=f'/result/{a_raw_key}')
            # a_stream_info = deepcopy(_out_xdf_stream_infos_df).loc[an_xdf_dataset_idx]    
            # print(f'i: {i}, a_meas_date: {a_meas_date}, a_stream_info: {a_stream_info}\n\n')
            # print(f'i: {an_xdf_dataset_idx}, a_meas_date: {a_meas_date}')
            # a_df = a_raw.annotations.to_data_frame(time_format='datetime')
            # a_df = a_df[a_df['description'] != 'BAD_motion']
            # a_df['xdf_dataset_idx'] = an_xdf_dataset_idx
            # flat_annotations.append(a_df)

# flat_annotations = pd.concat(flat_annotations, ignore_index=True)
# flat_annotations

In [ ]:
flat_annotations['onset_str'] = flat_annotations['onset'].dt.strftime("%Y-%m-%d_%I:%M:%S.%f %p")
flat_annotations

# Out to HDF5

In [ ]:
import h5py
from phoofflineeeganalysis.analysis.EEG_data import EEGComputations

hdf5_out_path: Path = Path('E:/Dropbox (Personal)/Databases/AnalysisData/MNE_preprocessed').joinpath('2025-09-29_all_HDF.h5').resolve()
hdf5_out_path


In [ ]:
import h5py

_loaded_dict = {'xdf_stream_infos_df': None, 'flat_annotations': None, }

_loaded_dict['xdf_stream_infos_df'] = pd.read_hdf(hdf5_out_path, mode='r', key='/xdf_stream_infos_df')
_loaded_dict['flat_annotations'] = pd.read_hdf(hdf5_out_path, mode='r', key='/flat_annotations')
_loaded_dict


In [ ]:
with h5py.File(hdf5_out_path, 'r') as f:
    # f.attrs
    # f.name
    # f.visititems()
    _loaded_dict['xdf_stream_infos_df'] = pd.read_hdf(f, key='/xdf_stream_infos_df')
    

In [ ]:
xdf_stream_infos_df.to_hdf(hdf5_out_path, key='/xdf_stream_infos_df', append=False)
flat_annotations.to_hdf(hdf5_out_path, key='/flat_annotations_df', append=True)


In [ ]:
for an_xdf_dataset_idx in np.arange(num_sessions):
    a_raw = active_only_out_eeg_raws[an_xdf_dataset_idx]
    a_meas_date: datetime = a_raw.info.get('meas_date')
    a_result = results[an_xdf_dataset_idx]
    # a_stream_info = deepcopy(_out_xdf_stream_infos_df).loc[an_xdf_dataset_idx]    
    # print(f'i: {i}, a_meas_date: {a_meas_date}, a_stream_info: {a_stream_info}\n\n')
    # print(f'i: {an_xdf_dataset_idx}, a_meas_date: {a_meas_date}')
    # a_df = a_raw.to_data_frame(time_format='datetime')

    a_raw_key: str = a_meas_date.strftime("%Y-%m-%d/%H-%M-%S") # '2025-09-22/21-35-47'
    a_raw.to_data_frame(time_format='datetime').to_hdf(hdf5_out_path, key=f'/raw/{a_raw_key}/df', append=True)
    EEGComputations.to_hdf(a_result=a_result, file_path=hdf5_out_path, root_key=f'/result/{a_raw_key}')
    


In [ ]:
# a_raw_key: str = a_meas_date.strftime("%Y-%m-%dT%H-%M-%S")
a_raw_key: str = a_meas_date.strftime("%Y-%m-%d/%H-%M-%S") # '2025-09-22/21-35-47'
a_raw.to_data_frame(time_format='datetime').to_hdf(hdf5_out_path, key=f'/raw/{a_raw_key}/df', append=True)

EEGComputations.to_hdf(a_result=a_result, file_path=hdf5_out_path, root_key=f'/result/{a_raw_key}')



In [ ]:



all_WHISPER_df.drop(columns=['filepath']).to_hdf(hdf5_out_path, key='modalities/WHISPER/df', append=True)
all_pho_log_to_lsl_df.drop(columns=['filepath']).to_hdf(hdf5_out_path, key='modalities/PHO_LOG_TO_LSL/df', append=True)


# Spike3DRasterWindowWidget

In [ ]:
# from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.SpikeRasterWidgets.Spike2DRaster import Spike2DRaster
# from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import _setup_spike_raster_window_for_debugging
# from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Mixins.Render2DScrollWindowPlot import ScatterItemData
from pyphoplacecellanalysis.GUI.Qt.SpikeRasterWindows.Spike3DRasterWindowWidget import Spike3DRasterWindowWidget


In [ ]:
## It's passed a specific computation_result which has a .sess attribute that's used to determine which spikes are displayed or not.
spike_raster_window: Spike3DRasterWindowWidget = Spike3DRasterWindowWidget(spikes_df, type_of_3d_plotter=type_of_3d_plotter, application_name=f'Spike Raster Window - {active_display_fn_identifying_ctx_string}', neuron_colors=neuron_colors, neuron_sort_order=neuron_sort_order,
                                                                            params_kwargs=dict(use_docked_pyqtgraph_plots=use_docked_pyqtgraph_plots),
                                                                            ) ## surprisingly only needs spikes_df !!?!
# Set Window Title Options:
a_file_prefix = str(computation_result.sess.filePrefix.resolve())
spike_raster_window.setWindowFilePath(a_file_prefix)
spike_raster_window.setWindowTitle(f'Spike Raster Window - {active_config_name} - {a_file_prefix}')


In [ ]:

# Gets the existing SpikeRasterWindow or creates a new one if one doesn't already exist:
# spike_raster_window, (active_2d_plot, active_3d_plot, main_graphics_layout_widget, main_plot_widget, background_static_scroll_plot_widget) = Spike3DRasterWindowWidget.find_or_create_if_needed(curr_active_pipeline, force_create_new=True, allow_replace_hardcoded_main_plots_with_tracks=True)
# spike_raster_window, (active_2d_plot, active_3d_plot, main_graphics_layout_widget, main_plot_widget, background_static_scroll_plot_widget) = Spike3DRasterWindowWidget.find_or_create_if_needed(curr_active_pipeline, force_create_new=False, allow_replace_hardcoded_main_plots_with_tracks=True)
# spike_raster_window, (active_2d_plot, active_3d_plot, *_all_outputs_dict) = Spike3DRasterWindowWidget.find_or_create_if_needed(curr_active_pipeline, force_create_new=False, allow_replace_hardcoded_main_plots_with_tracks=True)
spike_raster_window, (active_2d_plot, active_3d_plot, *_all_outputs_dict) = Spike3DRasterWindowWidget.find_or_create_if_needed(curr_active_pipeline, force_create_new=True, allow_replace_hardcoded_main_plots_with_tracks=True)

# CustomCalendarWidget

In [ ]:
from datetime import datetime, timezone
from phoofflineeeganalysis.UI.CustomCalendarWidget import CalendarDatasource
from phoofflineeeganalysis.UI.CustomCalendarWidget import CustomDataDisplayingCalendar

a_ds: CalendarDatasource = CalendarDatasource(xdf_stream_infos_df=xdf_stream_infos_df)
ex = CustomDataDisplayingCalendar()
ex.show()
ex.set_datasource(data_source=a_ds)

In [ ]:
found_df = a_ds.get_records_for_day(day_date=datetime(2025, 9, 10, tzinfo=timezone.utc))
found_df

In [ ]:
day_date = datetime(2025, 9, 11, tzinfo=timezone.utc)
day_date = day_date.replace(hour=0, minute=0, second=0, microsecond=0).astimezone(tz=timezone.utc)
day_date



# Direct Analysis

In [ ]:
from pyphocorehelpers.plotting.image_plotting_helpers import IMShowHelpers



def plot_matrix(xbin_edges, ybin_edges, matrix, ax, **kwargs):

    def setup_stable_axes_limits(xbins_edges, ybin_edges, ax):
        " manually sets the axis data limits to disable autoscaling given the xbins_edges/ybin_edges "
        # x == horizontal orientation:
        ax.set_xlim(left=xbins_edges[0], right=xbins_edges[-1])
        ax.set_ylim(bottom=ybin_edges[0], top=ybin_edges[-1])


    variable_value = matrix

    xmin, xmax, ymin, ymax = (xbin_edges[0], xbin_edges[-1], ybin_edges[0], ybin_edges[-1]) # the same for both orientations
    x_first_extent = (xmin, xmax, ymin, ymax) # traditional order of the extant axes
    # y_first_extent = (ymin, ymax, xmin, xmax) # swapped the order of the extent axes.
    main_plot_kwargs = {
        'cmap': 'viridis',
        'origin':'lower',
        'extent':x_first_extent,
        'aspect': 'auto',        
    }

    """
    Note that changing the origin while keeping everything else the same doesn't flip the direction of the yaxis labels despite flipping the yaxis of the data.
    """
    im_out = ax.imshow(variable_value, **main_plot_kwargs)
    setup_stable_axes_limits(xbin_edges, ybin_edges, ax)
    return im_out




In [ ]:
fig = plt.figure(layout="constrained", figsize=[9, 9], dpi=220, clear=True) # figsize=[Width, height] in inches.
long_width_ratio = 1
ax_dict = fig.subplot_mosaic(
    [
        ["ax_dumb", "ax_dumb_avg"],
        ["ax_good", "ax_good_avg"],
		
        # ["ax_dumb_avg"],
        # ["ax_good_avg"],
    ],
    # set the height ratios between the rows
    # set the width ratios between the columns
    width_ratios=[10, 1],
    sharey=True,
    gridspec_kw=dict(wspace=0, hspace=0.0) # `wspace=0`` is responsible for sticking the pf and the activity axes together with no spacing
)
fig.show()

In [ ]:
import napari

viewer = napari.Viewer(ndisplay=3)

# viewer.add_points(mov_avg_filtered, size=2, face_color='red', name='mov_avg_filtered')
# viewer.add_points(data2, size=2, face_color='green', name='dataset2')
# viewer.add_points(data3, size=2, face_color='blue', name='dataset3')



In [ ]:

named_sessions_dict = {'dumb': -2,
					   'good': -6,
}

# dumb_session_idx: int = -2

_out_layers = {}

for a_name, an_xdf_dataset_idx in named_sessions_dict.items():
    a_raw = active_only_out_eeg_raws[an_xdf_dataset_idx]
    a_meas_date: datetime = a_raw.info.get('meas_date')
    a_raw_key: str = a_meas_date.strftime("%Y-%m-%d/%H-%M-%S") # '2025-09-22/21-35-47'

    # an_annotations_df = a_raw.annotations.to_data_frame(time_format='datetime')
    # an_annotations_df = an_annotations_df[an_annotations_df['description'] != 'BAD_motion']

    # a_result = results[an_xdf_dataset_idx]
    # # a_stream_info = deepcopy(_out_xdf_stream_infos_df).loc[an_xdf_dataset_idx]    
    # Sxx = a_result['spectogram']['Sxx']
    # Sxx_avg = a_result['spectogram']['Sxx_avg']
    # # t = a_result['spectogram']['t']
    # # freqs = a_result['spectogram']['freqs']
    # # fs = a_result['spectogram']['fs']
    # freqs, t, _ = a_result['spectogram']['spectogram_result_dict']['AF3']
    
    # Sxx_avg_across_channel_avg = np.atleast_2d(Sxx_avg.mean(dim='channels', skipna=True))
    # # Sxx_avg_across_channel_avg
    
    # Sxx_across_channel_avg = Sxx.mean(dim='channels', skipna=True)
    # # Sxx_across_channel_avg

    # ax_label = f"ax_{a_name}"
    # ax_label_avg = f"ax_{a_name}_avg"
    # # np.shape(Sxx_across_channel_avg)
    
    # # xbin = deepcopy(t)
    # # ybin = deepcopy(freqs)
    # # xmin, xmax, ymin, ymax = (xbin[0], xbin[-1], ybin[0], ybin[-1])
    # # # xmin, xmax, ymin, ymax = (active_one_step_decoder.ybin[0], active_one_step_decoder.ybin[-1], active_one_step_decoder.xbin[0], active_one_step_decoder.xbin[-1]) # Reversed x and y axes, seems not good.
    # # extent = (xmin, xmax, ymin, ymax)
    
    # # ax_dict[ax_label].imshow(Sxx_avg_across_channel_avg.T)
    # # ax_dict[ax_label_avg].imshow(Sxx_across_channel_avg, extent=extent, origin='lower')
    
    # # fig, axs, plot_im_out = IMShowHelpers.final_x_vertical_plot_imshow(xbin_edges=np.arange(1), ybin_edges=freqs, matrix=Sxx_avg_across_channel_avg, ax=ax_dict[ax_label])
    # # ax_dict[ax_label].autoscale(False)
    # # fig, axs, plot_im_out = IMShowHelpers.final_x_horizontal_plot_imshow(xbin_edges=t, ybin_edges=freqs, matrix=Sxx_across_channel_avg, ax=ax_dict[ax_label])
    # im_out = plot_matrix(xbin_edges=t, ybin_edges=freqs, matrix=Sxx_across_channel_avg, ax=ax_dict[ax_label])
    # ax_dict[ax_label].set_ylabel(a_name)
    # # ax_dict[ax_label].autoscale(False)
    

    # # fig, axs, plot_im_out = IMShowHelpers.final_x_vertical_plot_imshow(xbin_edges=t, ybin_edges=freqs, matrix=Sxx_avg_across_channel_avg, ax=ax_dict[ax_label_avg])
    # avg_im_out = plot_matrix(xbin_edges=[0.0, 1.0], ybin_edges=freqs, matrix=Sxx_avg_across_channel_avg, ax=ax_dict[ax_label_avg])
    # # ax_dict[ax_label_avg].set_ylabel(f"{a_name}_avg")

    
    # napari_img_layer_kwargs = dict(
    #     # channel_axis=None,
    #     # rgb=None,
    #     colormap='yellow', #'bop_blue',
    #     # contrast_limits=None,
    #     gamma=0.02,
    #     interpolation2d='nearest',
    #     interpolation3d='linear',
    #     rendering='mip',
    #     depiction='volume',
    #     # iso_threshold=None,
    #     # attenuation=0.05,
    #     # name=None,
    #     # metadata=None,
    #     # scale=None,
    #     translate=None,
    #     # rotate=None,
    #     # shear=None,
    #     # affine=None,
    #     # opacity=1,
    #     blending='additive',
    #     visible=True,
    #     # multiscale=None,
    #     # cache=True,
    #     # plane=None,
    #     # experimental_clipping_planes=None,
    #     # custom_interpolation_kernel_2d=None,
    # )
    

    # curr_name = f"{a_name}"
    # _out_layers[curr_name] = viewer.add_image(Sxx, name=curr_name, **napari_img_layer_kwargs)
    # viewer.dims.axis_labels = Sxx.dims # ('channels', 'freqs', 'times')

    # curr_name = f"{a_name}_avg"
    # # napari_img_layer_kwargs['translate'] = [0.0, 1852.0]
    # napari_img_layer_kwargs['translate'] = [0.0, -2.0]
    # _out_avg_layer = viewer.add_image(Sxx_avg.T, name=curr_name, **napari_img_layer_kwargs) #  colormap='bop_blue', gamma=0.20, rendering='additive'
    # _out_layers[curr_name] = _out_avg_layer


In [ ]:
# _out_avg_layer.translate([512.0, 0.0])
# _out_avg_layer.set_translation([512.0, 0.0])
# _out_avg_layer.translate = [0.0, 513.0]
_out_avg_layer.translate = [0.0, 1852.0]
_out_avg_layer.translate

In [ ]:
_out_avg_layer.

In [ ]:
viewer.dims
viewer.layers

In [ ]:
from pyphoplacecellanalysis.GUI.Napari.napari_helpers import napari_extract_layers_info

# @function_attributes(short_name=None, tags=['napari', 'config'], input_requires=[], output_provides=[], uses=[], used_by=['napari_extract_layers_info'], creation_date='2024-08-12 08:54', related_items=[])
def extract_layer_info(a_layer):
    """ Extracts info as a dict from a single Napari layer. 
    by default Napari layers print like: `<Shapes layer 'Shapes' at 0x1635a1e8460>`: without any properties that can be easily referenced.
    This function extracts a dict of properties.

    from pyphoplacecellanalysis.GUI.Napari.napari_helpers import extract_layer_info

    """
    out_properties_dict = {}
    positioning = ['scale', 'translate', 'rotate', 'shear', 'affine', 'corner_pixels']
    visual = ['opacity', 'blending', 'visible', 'z_index', 'contrast_limits_range', '_colormap_name', 'gamma']
    # positioning = ['scale', 'translate', 'rotate', 'shear', 'affine']
    out_properties_dict['positioning'] = {}

    for a_property_name in positioning:
        out_properties_dict['positioning'][a_property_name] = getattr(a_layer, a_property_name)

    out_properties_dict['visual'] = {}
    for a_property_name in visual:
        try:
            out_properties_dict['visual'][a_property_name] = getattr(a_layer, a_property_name)
        except (AttributeError, ValueError, KeyError, TypeError) as e:
            print(f'failed to get property: "{a_property_name}" with error {e}')
            pass
        except Exception as e:
            raise

    return out_properties_dict


# @function_attributes(short_name=None, tags=['napari', 'config'], input_requires=[], output_provides=[], uses=['extract_layer_info'], used_by=[], creation_date='2024-08-12 08:54', related_items=[])
def napari_extract_layers_info(layers):
	"""extracts info dict from each layer as well.
	Usage:
        from pyphoplacecellanalysis.GUI.Napari.napari_helpers import napari_extract_layers_info
		layers = directional_viewer.layers # [<Shapes layer 'Shapes' at 0x1635a1e8460>, <Shapes layer 'Shapes [1]' at 0x164d5402e50>]
		out_layers_info_dict = debug_print_layers_info(layers)

	"""
	out_layers_info_dict = {}
	for a_layer in layers:
		a_name: str = str(a_layer.name)
		out_properties_dict = extract_layer_info(a_layer)
		out_layers_info_dict[a_name] = out_properties_dict
		# if isinstance(a_layer, Shapes):
		# 	print(f'shapes layer: {a_layer}')
		# 	a_shapes_layer: Shapes = a_layer
		# 	# print(f'a_shapes_layer.properties: {a_shapes_layer.properties}')
		# 	out_properties_dict = extract_layer_info(a_layer)
		# 	print(f'\tout_properties_dict: {out_properties_dict}')
		# 	out_layers_info_dict[a_name] = out_properties_dict
		# else:
		# 	print(f'unknown layer: {a_layer}')	
	return out_layers_info_dict





layers = viewer.layers # [<Shapes layer 'Shapes' at 0x1635a1e8460>, <Shapes layer 'Shapes [1]' at 0x164d5402e50>]
out_layers_info_dict = napari_extract_layers_info(layers)
out_layers_info_dict

In [ ]:
out_layers_info_dict['dumb']

In [ ]:
from napari.layers.image.image import Image

property_names = ['metadata', 'blending', 'opacity', 'rendering', 'scale', 'gamma', 'contrast_limits_range', 'colormap']
for a_layer in layers:
    an_img_layer: Image = a_layer
    
#    type(a_layer)

# an_img_layer.blending
an_img_layer.__dict__

In [ ]:
napari.run()

In [ ]:
xbin
ybin

In [ ]:
a_raw.plot()

In [ ]:
a_stream_info

In [ ]:
a_raw

In [ ]:
import xarray as xr

## INPUTS: a_result
a_spectogram_result: Dict = a_result['spectogram'] 

ch_names = a_spectogram_result['ch_names']
fs = a_spectogram_result['fs']
a_spectogram_result_dict = a_spectogram_result['spectogram_result_dict'] # Dict[channel: Tuple]
Sxx = a_spectogram_result['Sxx']
Sxx_avg = a_spectogram_result['Sxx_avg']

Sxx

In [ ]:

Sxx_avg_list = [] 

# ch_names = a_raw.info.ch_names

for a_ch, a_tuple in a_spectogram_result_dict.items():
    f, t, Sxx = a_tuple ## unpack the tuple
    # np.shape(Sxx) # (513, 1116) - (n_freqs, n_times)
    n_freqs = np.shape(f)
    n_times = np.shape(t) 
    Sxx_avg = np.nanmean(Sxx, axis=-1) ## average over all time to get one per session
    Sxx_avg_list.append(Sxx_avg)
    
Sxx_avg_list = np.stack(Sxx_avg_list) # (14, 513) - (n_channels, n_freqs)
Sxx_avg_list = xr.DataArray(Sxx_avg_list, dims=("channels", "freqs"), coords={"channels": ch_names, "freqs": f})
np.shape(Sxx_avg_list)
Sxx_avg_list

In [ ]:
ch_names

In [ ]:
a_raw.annotations

In [ ]:
Sxx_avg = np.nanmean(Sxx, axis=-1) ## average over all time to get one per session
Sxx_avg

In [ ]:
plt.close('all')

In [ ]:
plot_all_spectograms(active_only_out_eeg_raws, results)

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams["axes.titlesize"] = 8
plt.rcParams["axes.labelsize"] = 8
plt.rcParams["xtick.labelsize"] = 6
plt.rcParams["ytick.labelsize"] = 6
plt.rcParams["legend.fontsize"] = 6
plt.rcParams["figure.titlesize"] = 8
plt.rcParams["axes.titlepad"] = 0
plt.rcParams["figure.constrained_layout.use"] = True
plt.rcParams["figure.constrained_layout.h_pad"] = 0.0
plt.rcParams["figure.constrained_layout.w_pad"] = 0.0
plt.rcParams["figure.constrained_layout.hspace"] = 0.0
plt.rcParams["figure.constrained_layout.wspace"] = 0.0
plt.rcParams["figure.subplot.wspace"] = 0.0
plt.rcParams["figure.subplot.hspace"] = 0.0
plt.rcParams["figure.subplot.wspace"] = 0.0
plt.rcParams["figure.subplot.hspace"] = 0.0


In [ ]:
## Plot a synchronized EEG Raw data and Spectogram Figure together:
active_eeg_idx: int = -4
mne_raw_fig = active_only_out_eeg_raws[active_eeg_idx].plot(time_format='datetime', scalings='auto') # MNEBrowseFigure
fig, axs = plot_session_spectogram(active_only_out_eeg_raws[active_eeg_idx], results[active_eeg_idx], sync_to_mne_raw_fig=mne_raw_fig)
# plt.subplots_adjust(wspace=0, hspace=0)  # remove spacing


In [ ]:
active_eeg_idx: int = -6
mne_raw_fig2 = active_only_out_eeg_raws[active_eeg_idx].plot(time_format='datetime', scalings='auto') # MNEBrowseFigure
fig2, axs2 = plot_session_spectogram(active_only_out_eeg_raws[active_eeg_idx], results[active_eeg_idx], sync_to_mne_raw_fig=mne_raw_fig)

### Compute and show all spectograms

In [ ]:
# Compute results first
active_only_out_eeg_raws, results = batch_compute_all_eeg_datasets(eeg_raws=_out_eeg_raw, limit_num_items=3)

# Render to PDFs (paged)
from pathlib import Path
out_paths = render_all_spectograms_to_high_quality_pdfs(
    active_only_out_eeg_raws,
    results,
    output_parent_folder=Path(r"E:/Dropbox (Personal)/Databases/AnalysisData/MNE_preprocessed/exports"),
    mode="paged",
    seconds_per_page=180.0,
    freq_min_hz=1.0,
    freq_max_hz=40.0,
    dpi=300
)
print(f"Wrote {len(out_paths)} PDF(s)")

## ANalysis for Fatigue/Bandpowers

In [ ]:
from phoofflineeeganalysis.analysis.computations.fatigue_analysis import compare_multiple_recordings, compute_fatigue_metrics, analyze_fatigue_trends, print_analysis_report, visualize_fatigue_comparison

raw_objects = deepcopy(_out_eeg_raw)
raw_obj_labels = [str(a_raw) for a_raw in raw_objects]
if len(raw_objects) >= 2:
    results = compare_multiple_recordings(raw_objects, raw_obj_labels[:len(raw_objects)])
    print_analysis_report(results)
    visualize_fatigue_comparison(results)

    results
else:
    print("Not enough data files found for comparison.")


## Sliding wavlet analyses

In [ ]:
import pyqtgraph as pg
from phoofflineeeganalysis.timeflux.nodes.wavelet_cwt import EEGViewer

app = pg.mkQApp('EEGWaveletViewer')

raw = _out_eeg_raw[-1]

# Launch the Qt Application
viewer = EEGViewer(raw_data=raw)
viewer.show()
# sys.exit(app.exec_())


In [ ]:
fixed_len_epochs

In [ ]:
from phoofflineeeganalysis.timeflux.nodes.wavelet_cwt import plot_raw_with_cwt

# --- Example Usage ---

# First, create a sample 14-channel Raw object for demonstration
# sfreq = 250
# ch_names = [f'EEG {i+1:02d}' for i in range(14)]
# info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types='eeg')
# n_seconds = 120
# data = np.random.randn(len(ch_names), sfreq * n_seconds)
# raw = mne.io.RawArray(data, info)

raw = _out_eeg_raw[-1]

# Now, use the function to plot the first channel (index 0) from 10 to 20 seconds
plot_raw_with_cwt(raw, start_seconds=10.0, end_seconds=20.0, channel_index=0)
